In [ ]:
import sys
import os
sys.path.append(os.environ['GOTMWORK_ROOT']+'/tools', )
from gotmanalysis import *
np.seterr(all='raise')
%matplotlib inline

In [ ]:
timetag = '20080701-20080731'
casename = 'JRA55-do_Global'
forcing_reg_type = 'BG12'
tmname = 'KPP-CVMix'
update_data = False
plot_figure = True
apply_mask = True

In [ ]:
# check forcing_reg_type
fr_list = ['BG12', 'LF17']
if forcing_reg_type not in fr_list:
    print('Forcing regime {} not supported. Stop.'.format(forcing_reg_type))

In [ ]:
# check time tag
month_labels = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
timetag_list = ['20090101-20090131',
                '20090201-20090228',
                '20090301-20090331',
                '20090401-20090430',
                '20090501-20090531',
                '20080601-20080630',
                '20080701-20080731',
                '20080801-20080831',
                '20080901-20080930',
                '20081001-20081031',
                '20081101-20081130',
                '20081201-20081231']
try:
    idx = timetag_list.index(timetag)
except ValueError:
    print('Time tag {} not supported. Stop.'.format(timetag))
    
mlabel = month_labels[idx]

In [ ]:
# paths
s1data_root = os.environ['GOTMRUN_ROOT']+'/'+casename+'/VR1m_DT600s_'+timetag
s2data_root = os.environ['GOTMFIG_ROOT']+'/data/'+casename+'/VR1m_DT600s_'+timetag
fig_root = os.environ['GOTMFIG_ROOT']+'/'+casename+'/VR1m_DT600s_'+timetag
os.makedirs(s2data_root, exist_ok=True)
os.makedirs(fig_root, exist_ok=True)

In [ ]:
# get forcing regime
basepath = s1data_root+'/'+tmname
s2data_name = s2data_root+'/data_forcing_regime_'+forcing_reg_type+'_'+tmname+'.npz'
mask_name = s2data_root+'/mask_'+tmname+'.npz'
if update_data or not os.path.isfile(s2data_name):
    # update data
    print('Updating data...')
    loclist = sorted(os.listdir(basepath))
    pathlist = [basepath+'/'+x+'/gotm_out_s1.nc' for x in loclist]
    godmobj = GOTMOutputDataMap(pathlist)
    forcing_regime = np.zeros(godmobj.ncase)
    for i in np.arange(godmobj.ncase):
        if np.mod(i, 100) == 0:
            print('{:6.2f} %'.format(i/godmobj.ncase*100.0))
        tmp = GOTMOutputData(godmobj._paths[i], init_time_location=False)
        if forcing_reg_type == 'BG12':
            forcing_regime[i] = tmp.diag_forcing_regime_BG12()
        elif forcing_reg_type == 'LF17':
            forcing_regime[i] = tmp.diag_forcing_regime_LF17()

    gmobj = GOTMMap(data=forcing_regime, lon=godmobj.lon, lat=godmobj.lat, name='forcing_regime')
    if apply_mask:
        # read mask
        gmobj_mask = GOTMMap().load(mask_name)
        # apply mask
        gmobj.masked(gmobj_mask)
    # save data
    gmobj.save(s2data_name)
else:
    # read data
    gmobj = GOTMMap().load(s2data_name)
    if apply_mask:
        # read mask
        gmobj_mask = GOTMMap().load(mask_name)
        # apply mask
        gmobj.masked(gmobj_mask)
        # update data
        gmobj.save(s2data_name)

In [ ]:
# plot figure
if plot_figure:
    # create figure
    fig = plt.figure()
    fig.set_size_inches(6, 2.2)
    # plot
    plot_forcing_regime(gmobj)
    # label
    plt.text(40, 55, mlabel, fontsize=14, color='k', fontweight='bold', va='top',
            bbox=dict(boxstyle="square",ec='k',fc='w'))
    # reduce margin
    plt.tight_layout()
    # save figure
    figname = fig_root+'/fig_forcing_regime_'+forcing_reg_type+'.png'
    plt.savefig(figname, dpi = 300)